In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def scrap(año, mes):
    url  = 'https://www.cotizacion-dolar.com.ar/dolar-blue-historico-'+str(año)+'.php'
    for i in range(1,7):
        try:        
            fecha = datetime.datetime(año,mes,i)
            data = {'fecha': fecha.strftime('%d-%m-%y')}
            resp = requests.post(url, data=data)
            soup = BeautifulSoup(resp.text, "html.parser")
            break
        except:
            print('Falló en ',i)    
    filas = soup.find_all('td', {'style' : 'padding: 1%'})
    return filas

def parsear(filas):
    mensual = pd.DataFrame() 
    for i in range(1, int(len(list(filas))/3)):
        dic = {}
        dic['fecha'] = filas[3*i].text
        dic['bid'] = filas[3*i+1].text
        dic['ask'] = filas[3*i+2].text
        rueda = pd.DataFrame.from_dict(dic, orient='index').transpose().set_index('fecha')
        rueda.index = pd.to_datetime(rueda.index, format='%d-%m-%y ')
        mensual = pd.concat([mensual,rueda], axis=0)
    return mensual

def downloadAño(año):
    tablaAnual = pd.DataFrame()
    for i in range(1,13):
        filas = scrap(año=año, mes=i)
        tabla = parsear(filas)
        tablaAnual = pd.concat([tablaAnual,tabla],axis=0)
        print('mes',i,'listo')        
    tablaAnual.to_excel('blue_'+str(año)+'.xlsx')
    print(tablaAnual)

downloadAño(2016)

mes 1 listo
mes 2 listo
mes 3 listo
mes 4 listo
mes 5 listo
mes 6 listo
mes 7 listo
mes 8 listo
mes 9 listo
mes 10 listo
mes 11 listo
mes 12 listo


ModuleNotFoundError: No module named 'openpyxl'

In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def scrap(año, mes):
    url = 'https://www.cotizacion-dolar.com.ar/dolar-blue-historico-'+str(año)+'.php'
    for i in range(1, 7):
        try:        
            fecha = datetime.datetime(año, mes, i)
            data = {'fecha': fecha.strftime('%d-%m-%y')}
            resp = requests.post(url, data=data)
            soup = BeautifulSoup(resp.text, "html.parser")
            break
        except:
            print('Falló en ', i)    
    filas = soup.find_all('td', {'style': 'padding: 1%'})
    return filas

def parsear(filas):
    datos_mensuales = []
    for i in range(1, int(len(list(filas)) / 3)):
        dic = {
            'fecha': filas[3 * i].text,
            'bid': filas[3 * i + 1].text,
            'ask': filas[3 * i + 2].text
        }
        datos_mensuales.append(dic)
    return datos_mensuales

def downloadAño(año):
    tablaAnual = []
    for i in range(1, 13):
        filas = scrap(año=año, mes=i)
        datos_mensuales = parsear(filas)
        tablaAnual.extend(datos_mensuales)
        print('mes', i, 'listo')        

    df_tablaAnual = pd.DataFrame(tablaAnual)
    df_tablaAnual['fecha'] = pd.to_datetime(df_tablaAnual['fecha'], format='%d-%m-%y')
    df_tablaAnual.to_csv('blue_'+str(año)+'.csv', index=False)
    print(df_tablaAnual)

downloadAño(2016)


mes 1 listo
mes 2 listo
mes 3 listo
mes 4 listo
mes 5 listo
mes 6 listo
mes 7 listo
mes 8 listo
mes 9 listo
mes 10 listo
mes 11 listo
mes 12 listo


ValueError: unconverted data remains when parsing with format "%d-%m-%y": " ", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.